In [2]:
# optional 
# AeroMind — Experimental Analysis
#
# This notebook analyzes logged data from the AeroMind system.
# It does NOT control the drone.
# It is used only for research evaluation, metrics, and plots.
#
# The goal is to:
# - Evaluate gesture recognition accuracy
# - Measure latency
# - Assess safety behavior
# - Support or reject research hypotheses (H1–H3)


In [3]:
# Core data analysis libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Make plots readable and clean
sns.set(style="whitegrid")


In [5]:
# Load one experimental run log
# Change filename for each experiment
log_path = "../data/logs/run_20260207_010135.csv"

df = pd.read_csv(log_path)

# Preview data structure
df.head()


,run_id,ts_ms,event_type,frame_id,participant_id,lighting,background,distance_m,gesture_true,gesture_pred,...,command_sent,command_block_reason,drone_state,battery_pct,height_cm,command_ts_ms,ack_ts_ms,drone_motion_ts_ms,e2e_latency_ms,notes
0,20260207_010135,1770447724544,frame,0,P1,normal,clean,0.5,NaN,none,...,none,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20260207_010135,1770447725824,frame,1,P1,normal,clean,0.5,NaN,none,...,none,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20260207_010135,1770447725855,frame,2,P1,normal,clean,0.5,NaN,none,...,none,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20260207_010135,1770447725874,frame,3,P1,normal,clean,0.5,NaN,none,...,none,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20260207_010135,1770447725900,frame,4,P1,normal,clean,0.5,NaN,none,...,none,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Frame-level data: predictions every frame
frames = df[df["event_type"] == "frame"]

# Command-level data: when a command was actually sent
commands = df[df["event_type"] == "command"]

# Convert numeric fields safely
frames["confidence"] = pd.to_numeric(frames["confidence"], errors="coerce")
frames["stable_ms"] = pd.to_numeric(frames["stable_ms"], errors="coerce")


In [7]:
# Use only labeled trials (gesture_true must be filled during experiments)
valid = frames[frames["gesture_true"] != ""]

# Accuracy = correct predictions / total predictions
accuracy = (valid["gesture_pred"] == valid["gesture_true"]).mean()

accuracy


np.float64(0.0)

In [8]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Build confusion matrix
cm = confusion_matrix(
    valid["gesture_true"],
    valid["gesture_pred"],
    labels=sorted(valid["gesture_true"].unique())
)

# Visualize errors between gestures
disp = ConfusionMatrixDisplay(
    cm,
    display_labels=sorted(valid["gesture_true"].unique())
)
disp.plot(xticks_rotation=45)
plt.title("Gesture Confusion Matrix")
plt.show()


c:\Users\jihad\Documents\MEGA\Classes\CS420\aeromind\venv\Lib\site-packages\sklearn\externals\array_api_compat\numpy\_aliases.py:125: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype=dtype, copy=copy)


ValueError: Input y_true contains NaN.

In [ ]:
# Compute accuracy per participant
acc_by_user = (
    valid.groupby("participant_id")
    .apply(lambda x: (x["gesture_pred"] == x["gesture_true"]).mean())
)

# Plot variation across users
acc_by_user.plot(kind="bar")
plt.ylabel("Accuracy")
plt.title("Gesture Accuracy per Participant")
plt.show()


In [ ]:
# False positive = system predicts a gesture when none was intended
frames["false_positive"] = (
    (frames["gesture_pred"] != "none") &
    (frames["gesture_true"] == "none")
)

# Compute false positive rate per lighting condition
fp_rate = frames.groupby("lighting")["false_positive"].mean()

fp_rate.plot(kind="bar")
plt.ylabel("False Positive Rate")
plt.title("False Positives vs Lighting")
plt.show()


In [ ]:
# Extract latency values (ms)
lat = commands["e2e_latency_ms"].dropna().astype(float)

# Plot distribution
plt.hist(lat, bins=30)
plt.xlabel("Latency (ms)")
plt.ylabel("Count")
plt.title("End-to-End Latency Distribution")
plt.show()

# Summary statistics
lat.describe()


In [ ]:
# Count reasons commands were blocked
block_counts = frames["command_block_reason"].value_counts()

block_counts.plot(kind="bar")
plt.ylabel("Count")
plt.title("Reasons Commands Were Blocked")
plt.show()


In [ ]:
# Count emergency stop commands
emergency_cmds = commands[commands["command_sent"] == "emergency"]

len(emergency_cmds)


In [ ]:
print("H1 (user variability):", acc_by_user.std() > 0)
print("H2 (lighting affects FP):",
      fp_rate.get("low", 0) > fp_rate.get("normal", 0))
print("H3 (safety blocks commands):",
      "low_confidence" in block_counts.index)
